In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

group_size = 50000 #max block size

query = ("https://data.nola.gov/resource/w68y-xmk6.json?"
         "$select=type_,timedispatch,timearrive,disposition,timecreate,location,policedistrict,priority"
         #columns of interest 
         "&$limit=" + str(group_size) +
         "&$order=nopd_item"
         "&$offset=")

group_id = 0
df_list = []
condition = True
while condition:
    the_offset = group_id * group_size
    df_list.append(pd.read_json(query + str(the_offset))) #loading data frame in chuncks
    group_id += 1
    condition = not df_list[-1].empty #until full database is taken

df2015 = pd.concat(df_list, ignore_index=True) # building the dataframe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432878 entries, 0 to 432877
Data columns (total 8 columns):
disposition       432878 non-null object
location          432878 non-null object
policedistrict    432878 non-null int64
priority          432877 non-null object
timearrive        338935 non-null object
timecreate        432878 non-null object
timedispatch      292661 non-null object
type_             432878 non-null object
dtypes: int64(1), object(7)
memory usage: 29.7+ MB


Q1:
    What fraction of calls are of the most common type?

In [2]:
print 'fraction of most common calls =', format(float(df2015['type_'].value_counts().max())/len(df2015),'.9f')

fraction of most common calls = 0.246351166


Q2:
Some calls result in an officer being dispatched to the scene, and some log an arrival time. What is the median response time (dispatch to arrival), in seconds, considering only valid (i.e. non-negative) times?

In [3]:
times_df = df2015.dropna()[['timearrive', 'timedispatch']].applymap(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))
# transforming strings into datetime

           timearrive        timedispatch
0 2015-01-01 01:41:20 2015-01-01 01:24:47
3 2015-01-01 00:13:19 2015-01-01 00:08:17
7 2015-01-01 00:13:46 2015-01-01 00:06:15
8 2015-01-01 00:09:18 2015-01-01 00:07:12


In [5]:
def timeDif(row): # calculating the difference row by row
    return (row['timearrive'] - row['timedispatch']).total_seconds()
deltas = times_df.apply(timeDif, axis=1)
deltas = deltas[deltas > 0]

In [10]:
mean_response_time = deltas.mean(axis=0)
print 'average response time =', format(mean_response_time, '.7f')

average response time = 496.1900981


Q3: Work out the average (mean) response time in each district. What is the difference between the average response times of the districts with the longest and shortest times?

In [11]:
deltas = pd.concat([deltas, df2015.ix[deltas.index,'policedistrict']], axis=1) #merging Dt and districts
deltas.columns = ['deltaT', 'policedistrict']
avgtimes = deltas.groupby(['policedistrict']).mean()
maxdelta = max(avgtimes['deltaT']) - min(avgtimes['deltaT'])
print 'difference in average response time =', format(maxdelta, '.7f')

difference in average response time = 445.3391748


Q4:
We can define surprising event types as those that occur more often in a district than they do over the whole city. What is the largest ratio of the conditional probability of an event type given a district to the unconditional probably of that event type? Consider only events types which have more than 100 events. Note that some events have their locations anonymized and are reported as being in district "0". These should be ignored.

In [12]:
surprising_df = df2015.ix[df2015['policedistrict'] != 0,['type_','policedistrict']] #looking at types in distr. 1 to 8
tot_crimes = surprising_df['type_'].count()

In [19]:
t_counts = surprising_df['type_'].value_counts()
freq_types = t_counts[t_counts > 100].index # selecting types with 100+ entries
p_counts =  surprising_df['policedistrict'].value_counts()
p_probs = p_counts / tot_crimes #calculating probability of districts

In [14]:
prob_list = []
for type_select in freq_types:        #preparing for prob frame
    for dist_select in p_counts.index:
        event_prob = float(t_counts[type_select]) / tot_crimes #event probability
        p_prob = float(p_counts[dist_select]) / tot_crimes # local prob
        intersect_prob = float(surprising_df.ix[(surprising_df['policedistrict'] == dist_select) & (surprising_df['type_'] == type_select), 'type_'].count()) / tot_crimes
        cond_prob = intersect_prob / p_prob
        prob_list.append([event_prob,p_prob,intersect_prob,cond_prob])        

In [18]:
probframe = pd.DataFrame(prob_list) #building frame
probframe.columns = ['ev_prob','dist_prob','inters_prob','cond_prob']
probframe['prob_ratio'] = probframe['cond_prob'] / probframe['ev_prob']
print 'max prob ratio of conditional/total prob =', probframe['prob_ratio'].max()

max prob ratio of conditional/total prob = 6.4630492074


Q5:
Find the call type that displayed the largest percentage decrease in volume between 2011 and 2015. What is the fraction of the 2011 volume that this decrease represents? The answer should be between 0 and 1.

In [28]:
# also for 2011 data we need do get df in chunks
query11 = ("https://data.nola.gov/resource/j7t8-jceh.json?"
           "$select=type"
           #columns of interest 
           "&$limit=" + str(group_size) +
           "&$order=nopd_item"
           "&$offset=")

group_id = 0
df_list = []
condition = True
while condition:
    the_offset = group_id * group_size
    df_list.append(pd.read_json(query11 + str(the_offset))) #loading data frame in chuncks
    group_id += 1
    condition = not df_list[-1].empty #until full database is taken

df2011 = pd.concat(df_list, ignore_index=True) # building the dataframe

In [33]:
type11 = df2011.groupby(['type'])['type'].count()
type15 = df2015.groupby(['type_'])['type_'].count()
max_decr_type = ((type11 - type15 )/ type11).argmax()
ratio_decr = ((type11 - type15 )/ type11).max()
print 'max decreased type =', max_decr_type, 'ratio w.r.t. 2011 = ', format(ratio_decr, '.9f')

max decreased type = 89 ratio w.r.t. 2011 =  0.994444444


Q6:
The disposition represents the action that was taken to address the serivce call. Consider how the disposition of calls changes with the hour of the record's creation time. Find the disposition whose fraction of that hour's disposition varies the most over a typical day. What is its change (maximum fraction minus minimum fraction)?

In [42]:
disp_df = pd.concat([df2015['disposition'], df2015['timecreate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S').hour)], axis=1)
disp_group = disp_df.groupby(['timecreate','disposition'])
disp_count = disp_group.disposition.count()
sumdisp = disp_count.sum(level='timecreate')
frac_disp = disp_count / sumdisp 
sw_sorted = frac_disp.swaplevel('timecreate','disposition').sortlevel(0)
print 'the disposition that varies the max during the day has a variation of =', (sw_sorted.max(level=0) - sw_sorted.min(level=0)).max()

the disposition that varies the max during the day has a variation of = 0.218860217348


Q7:
We can use the call locations to estimate the areas of the police districts. Represent each as an ellipse with semi-axes given by a single standard deviation of the longitude and latitude. What is the area, in square kilometers, of the largest district measured in this manner?

In [46]:
def val_select(row):
    if (not row.values()[1]) & (29.8 < float(row.values()[0]) < 30.21) & (-90.2 < float(row.values()[2]) < -89.5): 
        # a quick check at the map shows New Orleans is contained in 29.8-30.21 -90.2 - -89.5, eliminating bad entries
        row_mask.append(True)
    else: 
        row_mask.append(False)
    return #row.values()
    

In [49]:
row_mask = []
df2015.ix[:,'location'].apply(lambda x: val_select(x)) # getting the interesting rows
grouped = df2015.ix[row_mask,['location','policedistrict']].groupby('policedistrict')

def axis_to_area (axis_a, axis_b, center_a):
    Ravg = 6371 #in km
    latr = center_a * np.pi / 180
    D_latr = axis_a * np.pi / 180
    D_longr = axis_b * np.pi / 180
    # using Haversine formula for the great circle distance between 2 points
    # d_const_lat = 2R arcsin(cos(latr) abs(sin(D_longr/2))
    # d_const_long = R abs(D_latr)
    # R = 6371km (avg) Xr = pi/180 Xdeg 
    d_const_lat = 2 * Ravg * np.arcsin(np.cos(latr) * np.abs( np.sin(D_longr / 2) ) )
    d_const_long = Ravg * np.abs(D_latr)                   
    return np.pi * d_const_lat * d_const_long

def area_func (df, column='location'):
    axis_a = df[column].apply( lambda x: float(x.values()[0]) ).std()
    center_a = df[column].apply( lambda x: float(x.values()[0]) ).mean()
    axis_b = df[column].apply( lambda x: float(x.values()[2]) ).std()
    return axis_to_area(axis_a, axis_b, center_a)

areas = grouped.apply(area_func)
print 'the area in km^2 of the largest police district is', areas.max(), ' corresponding to district', areas.idxmax()

the area in km^2 of the largest police district is 24.9361986917  corresponding to district 7


Q8:
The calls are assigned a priority. Some types of calls will receive a greater variety of priorities. To understand which type of call has the most variation in priority, find the type of call whose most common priority is the smallest fraction of all calls of that type. What is that smallest fraction?

In [51]:
temp = df2015.groupby(['type_','priority']).priority.count()
sumtmp = temp.sum(level='type_')
min_fraction = (temp.max(level='type_') / sumtmp).min()
print 'the smallest fraction of calls of the most variate type =', min_fraction

the smallest fraction of calls of the most variate type = 0.475138121547
